### In this file, we will define our model and parameters

Instructions: 

Finally, you should have at least one notebook where you define and train your notebook.

In [23]:
import pandas as pd
import numpy as np
import torch
from dgl.data import DGLDataset
from dgl.data.utils import save_graphs, load_graphs

### Model

In [24]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

from dgl.nn.pytorch import NNConv

#### Ronan make your model here

In [ ]:
# Model

#### RJ model

In [65]:
# Things to experiment with: SumPooling layer? Kind of like a conv, can sum up neighbors and find important structural properties of molecule
# from dgl.nn import SumPooling

class Electron_MPNN(nn.Module):
    def __init__(self, node_in_feats, edge_in_feats, out_dim=1,
                 edge_hidden_feats=128):
        super(Electron_MPNN, self).__init__()

        self.fc1 = nn.Sequential(
            nn.Linear(node_in_feats, out_dim),
            nn.ReLU()
        )
    
        edge_func = nn.Sequential(
            nn.Linear(edge_in_feats, edge_hidden_feats),
            nn.ReLU(),
            nn.Linear(edge_hidden_feats, out_dim * out_dim)
        )
        self.gnn_layer = NNConv(
            in_feats=out_dim,
            out_feats=out_dim,
            edge_func=edge_func,
            aggregator_type='sum'
        )
        
        # GRU used in MPNN to get rid of vanishing gradient on Conv layer
        # However, I think it is just added complexity to our model, so we will keep it here for reference but not use it
#         self.gru = nn.GRU(out_dim, out_dim)
        
#         self.fc2 = nn.Linear(out_dim, out_dim)


    def forward(self, g, node_feats, edge_feats):
        node_feats = self.fc1(node_feats) # (V, node_out_feats)
        
        hidden_feats = node_feats.unsqueeze(0)           # (1, V, node_out_feats)

        node_feats = self.gnn_layer(g, node_feats, edge_feats)
        
#         for _ in range(self.num_step_message_passing):
#             node_feats = self.gnn_layer(g, node_feats, edge_feats)
            # GRU requires 3 dimensions
#             node_feats, hidden_feats = self.gru(node_feats.unsqueeze(0), hidden_feats)
#             node_feats = node_feats.squeeze(0)
            
#         node_feats = self.fc2(node_feats)
#         print(node_feats)
#         print("last_layer", node_feats)
        return node_feats.mean()

#### Load the graphs to get dimensions for our model

In [66]:
graphs, _ = load_graphs("./DataGraphs/data_F_graph.bin")

#### Construct the model

In [67]:
# All graphs in the list have the same scheme size, so pull the dimensions from the first
node_dim = graphs[0].ndata['atom_feats'].shape[1]
edge_dim = graphs[0].edata['bond_feats'].shape[1]
print("Dimensions:", node_dim, "(node),", edge_dim, "(edge)")

Dimensions: 11 (node), 5 (edge)


In [68]:
model = Electron_MPNN(node_dim, edge_dim)

In [69]:
sample_data = graphs[1]
edges = sample_data.edata['bond_feats'].float()
nodes = sample_data.ndata['atom_feats'].float()

In [70]:
model(sample_data, nodes, edges)

tensor(0.0091, grad_fn=<MeanBackward0>)

In [22]:
from os.path import exists
if exists('SavedModels/bestmae.txt'):
    with open('SavedModels/bestmae.txt', 'r') as f:
        txt = f.read()
        print(txt)
        print(float(txt))

123879.123
123879.123


In [21]:
best_mae = 123879.123
with open('SavedModels/bestmae.txt', 'w') as f:
    f.write(str(best_mae))